# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [14]:
# Write your code below.
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Retrieve PRICE_DATA path
price_data_path = os.getenv("PRICE_DATA")

# Ensure the environment variable is correctly set
if not price_data_path or not os.path.exists(price_data_path):
    raise ValueError("ERROR: PRICE_DATA environment variable is not set or the path does not exist.")

print(f"PRICE_DATA Path: {price_data_path}")

PRICE_DATA Path: ../../05_src/data/prices/


In [15]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [28]:
import os
from glob import glob

# Find all parquet files in the PRICE_DATA directory
parquet_files = glob(os.path.join(price_data_path, "**/*.parquet"), recursive=True)

# Ensure at least one file is found
if not parquet_files:
    raise ValueError(f"No parquet files found in {price_data_path}. Ensure you have executed `2_data_engineering.ipynb`.")

# Print first few files for verification
print(f"Found {len(parquet_files)} Parquet files:")
print(parquet_files[:5])  # Show first 5 files

Found 13078 Parquet files:
['../../05_src/data/prices/CTAS/CTAS_2002/part.0.parquet', '../../05_src/data/prices/CTAS/CTAS_2005/part.0.parquet', '../../05_src/data/prices/CTAS/CTAS_2004/part.0.parquet', '../../05_src/data/prices/CTAS/CTAS_2003/part.0.parquet', '../../05_src/data/prices/CTAS/CTAS_2010/part.0.parquet']


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
# Write your code below.

# Read all parquet files into a Dask DataFrame
df = dd.read_parquet(parquet_files, engine="pyarrow")

# Print column names for debugging
print("Original Columns:", df.columns)

# Rename 'Adj Close' to 'Adj_Close' if needed
if "Adj Close" in df.columns:
    df = df.rename(columns={"Adj Close": "Adj_Close"})

# Ensure 'Ticker' is a column (reset index if needed)
if "Ticker" not in df.columns:
    df = df.reset_index()

# Ensure required columns exist
required_columns = {"Ticker", "Close", "Adj_Close", "High", "Low", "Date"}
missing_cols = required_columns - set(df.columns)
if missing_cols:
    raise ValueError(f"ERROR: Missing required columns: {missing_cols}")

# Ensure Date is properly sorted for time series operations
df = df.sort_values(["Ticker", "Date"])

# Add lagged variables for Close and Adj_Close
df['Close_lag_1'] = df.groupby("Ticker")['Close'].shift(1, meta=("f8"))
df['Adj_Close_lag_1'] = df.groupby("Ticker")['Adj_Close'].shift(1, meta=("f8"))

# Calculate daily returns: (Close / Close_lag_1) - 1
df['returns'] = (df['Close'] / df['Close_lag_1']) - 1

# Calculate hi_lo_range: High - Low
df['hi_lo_range'] = df['High'] - df['Low']

# Assign the result to dd_feat
dd_feat = df

# Display first few rows
dd_feat.head()


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
df_pandas = dd_feat.compute()

# Add 10-day moving average of returns
df_pandas['returns_moving_avg_10'] = df_pandas.groupby("Ticker")['returns'].rolling(10).mean().reset_index(level=0, drop=True)

# Display the results
df_pandas.head()

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
Yes, because Dask does not support `.rolling()` operations across partitions. 
`.rolling(10).mean()` works well in Pandas but requires custom implementations in Dask.

+ Would it have been better to do it in Dask? Why?
For large datasets, using Dask's `map_partitions()` allows for distributed 
computation, making it more efficient for handling big data scenarios.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.